<h1>Algorithmic Trading Course</h1>

<h3>SECTION 01: Algorithmic Trading Basics</h3>

Algorithmic trading means using computers to mae invstment decisions.

There are many different types of algorithmic trading. The main difference is their spee of execution.

Here are some of the main players in the algorthmic trading landscape:
- Renaissance Technologies: $165Bin AUM
- AQR Capital Management: $61B in AUM
- Citadel Securities: $32B in AUM

<h4> The algorithmic Trading Process:</h4>
The process of running a quantitative investing strategy can be broken down into the following steps:<br>

1. Collect Data
2. Develope a hypohesis for a strategy
3. Backtest that strategy
4. Implement the strategy in production

<h3>SECTION 02: API Basics and Course Configuration</h3>

<h4> What is an API? </h4>
An API is an Application Programing Interface. It allow us to interact with someone else's software.<br>
We are going to use IEX Cloud API to access database of financial data.<br><br/>
In this course will be using the GET request to gather data from the IEX Cloud API.

Other API Functionality:
- POST: Adds data to the database exposed by the API. (Create only)
- PUT: Adds and verviewrites data in the database exposed by the API. (Create or replace)
- DELETE: Deletes data from the API's database.

----------

### PROJECT 01 - Equal-Weight S&P 500 Screener

The S&P 500 is the world's most popular stock market index.<br>
Many investment funds are benchmarked to the S&P 500. This means that they seek to replicate the performance of this index by owning all stocks that are held in the index.<br>
The S&P covers the 500 largest companies of United States. 

In the first project you will build an alternative version of the S&P 500 that assigns the same wheight to each company.<br>
The goal of this project is to create a Python script that accepts the value on your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

<h4>Importing libraries</h4>

In [23]:
import numpy as np
import pandas as pd 
import requests 
import xlsxwriter
import math

<h4> Importing Our List of Stocks</h4>
The stock in S&P 500 change over time, they are re-balanced.So in the real world you would connect to the Standard and Poors provider and pull their real-time database.<br></br>

For now we are going to use this static version of S&P. [Click this link to download them now](http://nickmccullum.com/algorithmic-trading-python/sp_500_stocks.csv). 

In [24]:
stocks = pd.read_csv('https://raw.githubusercontent.com/GustavoJannuzzi/Quant_projects_for_UPQuants/main/sp_500_stocks.csv')

<h4>Acquiring an API Token</h4>

API tokens an secret information must be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. 
For now we are going to use a sandbox mode, that's kinda like a game mode. It will make sure that all off ur API calls work, but it doesn't actually return real time financial data. It will return a random financial data. 

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [25]:
from secrets import IEX_CLOUD_API_TOKEN

<h4> Making the first API call</h4>
We 'll need the following information from the API:

- market capitalization for each stock
- price of each stock 

In [26]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 72819070,
 'calculationPrice': 'close',
 'change': 3.01,
 'changePercent': 0.01971,
 'close': 165.19,
 'closeSource': 'fliocaif',
 'closeTime': 1670440020431,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 161.343,
 'delayedPriceTime': 1671590493157,
 'extendedChange': 0.16,
 'extendedChangePercent': 0.00096,
 'extendedPrice': 157.82,
 'extendedPriceTime': 1742449475006,
 'high': 165.19,
 'highSource': '5ldi e eneyr i1ueacdpmt',
 'highTime': 1677996520943,
 'iexAskPrice': None,
 'iexAskSize': None,
 'iexBidPrice': None,
 'iexBidSize': None,
 'iexClose': 164.96,
 'iexCloseTime': 1734032571199,
 'iexLastUpdated': None,
 'iexMarketPercent': None,
 'iexOpen': 162.83,
 'iexOpenTime': 1729633431339,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexVolume': None,
 'lastTradeTime': 1707431320871,
 'latestPrice': 159.08,
 'latestSource': 'Close',
 'latestTime': 'September 9, 2022',
 'latestUpdate': 1719891896787,
 'latestVolume': 71205440,
 'low': 

<h4>Parsing Our API call</h4>
The API call that we executed in the last code block contains all the information required to build our equal-wieght S&P<br>



In [27]:
price = data['latestPrice']
market_cap = data['marketCap']

<h4>Adding stock data to Pandas DF</h4>

In [28]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to buy


In [31]:
final_dataframe = final_dataframe.append(
                                        pd.Series(['AAPL', 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to buy
0,AAPL,164.42,2598358347056,N/A
1,AAPL,157.45,2647291535773,N/A
2,AAPL,164.87,2616952206036,N/A
3,AAPL,157.71,2607874869721,N/A
4,AAPL,161.44,2590243636418,N/A
...,...,...,...,...
71,AAPL,162.35,2534575784981,N/A
72,AAPL,160.82,2583568106610,N/A
73,AAPL,160.42,2556211560537,N/A
74,AAPL,158.62,2614320196010,N/A


<h4> Looping throught the ticker in our list of stocks</h4>
Now we'll feed de dataframe calling all the stock in our list of stocks.

In [32]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbols in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)


In [ ]:
final_dataframe

<h4>Using Batch API Calls to improve Performance</h4>

<h4>Calculating the Number of share to buy</h4>

<h4>Formating our excel output</h4>

### PROJECT 02 - Quantitative Momentum Screener

<h3> Momentum Investing Project Overview</h3>
Momentum investing means investing in assets that have increased in price the most.</br>
Consider this example to better understand:<br>
Imagine that you have the choice between investing in two stocks that have had the following returns over the last year: 

- Apple(APPL): 35%
- Microsoft (MSFT): 20%

A momentum investing strategy would suggest investing in Apple because of its higher recent price return. <br>
There are many other nuances to momentum investing strategies.

### PROJECT 03 - Quantitative Value Investing Strategy Screener

<h3> Value Investing Project Overview</h3>
Value investing means investing in stocks that are trading below their perceived intrinsic value.<br>
Value investing was popularized by investors like Waren Buffet, seth Klarman, and  Benjamin Graham.<br></br>
Creating algorithmic value investing srtategies relies on a concept called <font color=red>multiples</font>.<br> 
Multiples are simply a way that investors use to estimate how valuable a company is.<br>
Multiples are calculated by dividing a company's stock price by sme measure of company's worth like earnings or assets.

Examples of <font color=red>multiples</font> used in value investing:

- Price-to-earnings
- Price-to-book-value
- Price-to-free-cash-flow

Each of the individual multilpes used by value investors has its pros and cons. <br>
One way to minimize the impact of any specific multiples is by using a <font color=red>composite</font>.<br></br>
Composite is just a average of many different valuation strategies. <br>
We'll use  composite of 5 different value investing metrics.